# Exploratory Data Analysis (EDA) with SQL  

## Introduction
Using this Python notebook you will:

1.  Understand the SpaceX Data Set
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 

In [ ]:
# Loads the sql extension to allow the usage of SQL directly within the notebook.
%load_ext sql

# Connects the notebook to the SQLite database my_data1.db
%sql sqlite:///my_data1.db

In [21]:
# Import the necessary libraries
import pandas as pd  # For data manipulation and analysis.
import csv, sqlite3  # For working with SQLite databases.

In [22]:
# Create a connection to the database
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

# Create a data frame
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

# Fetch the data for unique values
df['Landing_Outcome'].unique()

array(['Failure (parachute)', 'No attempt', 'Uncontrolled (ocean)',
       'Controlled (ocean)', 'Failure (drone ship)',
       'Precluded (drone ship)', 'Success (ground pad)',
       'Success (drone ship)', 'Success', 'Failure', 'No attempt '],
      dtype=object)

In [ ]:
# Display the data frame first 10 rows
df.head(10)

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt
5,2013-09-29,16:00:00,F9 v1.1 B1003,VAFB SLC-4E,CASSIOPE,500,Polar LEO,MDA,Success,Uncontrolled (ocean)
6,2013-12-03,22:41:00,F9 v1.1,CCAFS LC-40,SES-8,3170,GTO,SES,Success,No attempt
7,2014-01-06,22:06:00,F9 v1.1,CCAFS LC-40,Thaicom 6,3325,GTO,Thaicom,Success,No attempt
8,2014-04-18,19:25:00,F9 v1.1,CCAFS LC-40,SpaceX CRS-3,2296,LEO (ISS),NASA (CRS),Success,Controlled (ocean)
9,2014-07-14,15:15:00,F9 v1.1,CCAFS LC-40,OG2 Mission 1 6 Orbcomm-OG2 satellites,1316,LEO,Orbcomm,Success,Controlled (ocean)


In [24]:
# Identify which columns are numerical and categorical.
df.dtypes

Date                 object
Time (UTC)           object
Booster_Version      object
Launch_Site          object
Payload              object
PAYLOAD_MASS__KG_     int64
Orbit                object
Customer             object
Mission_Outcome      object
Landing_Outcome      object
dtype: object

In [25]:
# Identify the difeerent launch sites
df['Launch_Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [26]:
# Filter the data frame for the launch sites with the booster version, and payload mass.
df[['Booster_Version', 'Launch_Site', 'PAYLOAD_MASS__KG_']] 

,Booster_Version,Launch_Site,PAYLOAD_MASS__KG_
0,F9 v1.0 B0003,CCAFS LC-40,0
1,F9 v1.0 B0004,CCAFS LC-40,0
2,F9 v1.0 B0005,CCAFS LC-40,525
3,F9 v1.0 B0006,CCAFS LC-40,500
4,F9 v1.0 B0007,CCAFS LC-40,677
...,...,...,...
96,F9 B5B1062.1,CCAFS SLC-40,4311
97,F9 B5B1061.1,KSC LC-39A,12500
98,F9 B5B1063.1,VAFB SLC-4E,1192
99,F9 B5 B1049.7,CCAFS SLC-40,15600


In [32]:
# Remove blank rows from table if exists
%sql DROP TABLE IF EXISTS SPACEXTABLE;

# Create a table with the data from the SPACEXTBL table
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.
 * sqlite:///my_data1.db
Done.


[]

### EDA with SQL  

In [33]:
# Display the names of the unique launch sites  in the space mission
launch_site = cur.execute('SELECT DISTINCT Launch_Site FROM(SPACEXTABLE);')
pd.DataFrame(launch_site.fetchall(), columns=[x[0] for x in launch_site.description])

,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40


In [ ]:
# Display 5 records where launch sites begin with the string 'CCA' 
cur.execute("SELECT * FROM SPACEXTABLE WHERE Launch_Site LIKE 'CCA%' LIMIT 5;")

results = cur.fetchall()
for row in results:
    print(row)

('2010-06-04', '18:45:00', 'F9 v1.0  B0003', 'CCAFS LC-40', 'Dragon Spacecraft Qualification Unit', 0, 'LEO', 'SpaceX', 'Success', 'Failure (parachute)')
('2010-12-08', '15:43:00', 'F9 v1.0  B0004', 'CCAFS LC-40', 'Dragon demo flight C1, two CubeSats, barrel of Brouere cheese', 0, 'LEO (ISS)', 'NASA (COTS) NRO', 'Success', 'Failure (parachute)')
('2012-05-22', '7:44:00', 'F9 v1.0  B0005', 'CCAFS LC-40', 'Dragon demo flight C2', 525, 'LEO (ISS)', 'NASA (COTS)', 'Success', 'No attempt')
('2012-10-08', '0:35:00', 'F9 v1.0  B0006', 'CCAFS LC-40', 'SpaceX CRS-1', 500, 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')
('2013-03-01', '15:10:00', 'F9 v1.0  B0007', 'CCAFS LC-40', 'SpaceX CRS-2', 677, 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')


In [ ]:
# Display the total payload mass carried by boosters launched by NASA (CRS)

#  Total Payload Mass
cur.execute('SELECT SUM(PAYLOAD_MASS__KG_) AS TOTAL_PMK FROM SPACEXTABLE;')
results = cur.fetchone()
TOTAL_PMK = results[0]

# Quantity of Unique Payload Mass's
payload_mass = cur.execute('SELECT DISTINCT PAYLOAD_MASS__KG_ FROM SPACEXTABLE;')
payload_mass = pd.DataFrame(payload_mass)

# Display Results
print('Total Payload Mass:', TOTAL_PMK)
print('Quantity of Unique Values:', payload_mass.nunique())

# Display the first 10 rows of the payload mass
payload_mass.head(10)

Total Payload Mass: 619967
Quantity of Unique Values: 0    78
dtype: int64


,0
0,0
1,525
2,500
3,677
4,3170
5,3325
6,2296
7,1316
8,4535
9,4428


In [ ]:
# Display average payload mass carried by booster version F9 v1.1
F9_v11_Mean =cur.execute('SELECT AVG(PAYLOAD_MASS__KG_) FROM SPACEXTABLE WHERE Booster_Version = "F9 v1.1";')

# Display Results
print('F9 v1.1 Mean:', F9_v11_Mean.fetchone()[0])

F9 v1.1 Mean: 2928.4


In [ ]:
# List the date when the first successful landing outcome in ground pad was achieved.
success_landing = cur.execute('SELECT MIN(Date) FROM SPACEXTABLE WHERE Landing_Outcome = "Success";')

# Display the Results
print('First Successful Landing:', success_landing.fetchone()[0])

First Successful Landing: 2018-07-22


In [ ]:
# List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000
successful_busters = cur.execute('SELECT * FROM SPACEXTABLE WHERE Landing_Outcome =  "Success (drone ship)" AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000;')

# Display the Results
success_landing = pd.DataFrame(success_landing )
print(success_landing)

            0         1               2            3                      4  \
0  2016-05-06   5:21:00     F9 FT B1022  CCAFS LC-40               JCSAT-14   
1  2016-08-14   5:26:00     F9 FT B1026  CCAFS LC-40               JCSAT-16   
2  2017-03-30  22:27:00  F9 FT  B1021.2   KSC LC-39A                 SES-10   
3  2017-10-11  22:53:00  F9 FT  B1031.2   KSC LC-39A  SES-11 / EchoStar 105   

      5    6                       7        8                     9  
0  4696  GTO  SKY Perfect JSAT Group  Success  Success (drone ship)  
1  4600  GTO  SKY Perfect JSAT Group  Success  Success (drone ship)  
2  5300  GTO                     SES  Success  Success (drone ship)  
3  5200  GTO            SES EchoStar  Success  Success (drone ship)  


In [ ]:
# List the total number of successful and failure mission outcomes
total_landing_success = cur.execute('SELECT Landing_Outcome, COUNT(*) FROM SPACEXTABLE GROUP BY Landing_Outcome;')

# Display the Results
total_landing_success = pd.DataFrame(total_landing_success)
total_landing_success

,0,1
0,Controlled (ocean),5
1,Failure,3
2,Failure (drone ship),5
3,Failure (parachute),2
4,No attempt,21
5,No attempt,1
6,Precluded (drone ship),1
7,Success,38
8,Success (drone ship),14
9,Success (ground pad),9


In [ ]:
# List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery
boosterVersion_maxPayload = cur.execute('SELECT Booster_Version, PAYLOAD_MASS__KG_ FROM SPACEXTABLE WHERE PAYLOAD_MASS__KG_ = (SELECT MAX (PAYLOAD_MASS__KG_) FROM SPACEXTABLE);')

# Display the Results
boosterVersion_maxPayload = pd.DataFrame(boosterVersion_maxPayload)
boosterVersion_maxPayload


,0,1
0,F9 B5 B1048.4,15600
1,F9 B5 B1049.4,15600
2,F9 B5 B1051.3,15600
3,F9 B5 B1056.4,15600
4,F9 B5 B1048.5,15600
5,F9 B5 B1051.4,15600
6,F9 B5 B1049.5,15600
7,F9 B5 B1060.2,15600
8,F9 B5 B1058.3,15600
9,F9 B5 B1051.6,15600


In [ ]:
# List the records which will display the month names, failure landing_outcomes in drone ship,
# booster versions, launch_site for the months in year 2015.
failure_DroneShip = cur.execute(' SELECT * FROM SPACEXTABLE WHERE Landing_Outcome = "Failure (drone ship)" AND substr(Date, 6,2) AND substr(Date, 0,5)="2015";')

# Display the Results
failure_DroneShip = pd.DataFrame(failure_DroneShip)
failure_DroneShip

,0,1,2,3,4,5,6,7,8,9
0,2015-01-10,9:47:00,F9 v1.1 B1012,CCAFS LC-40,SpaceX CRS-5,2395,LEO (ISS),NASA (CRS),Success,Failure (drone ship)
1,2015-04-14,20:10:00,F9 v1.1 B1015,CCAFS LC-40,SpaceX CRS-6,1898,LEO (ISS),NASA (CRS),Success,Failure (drone ship)


In [ ]:
# Rank the count of landing outcomes (such as Failure (drone ship) or 
# Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.
landing_outcomes = cur.execute('SELECT Landing_Outcome, COUNT(*) as outcome_count  FROM SPACEXTABLE WHERE Date BETWEEN "2010-06-04" and "2017-03-30" GROUP BY Landing_Outcome ORDER BY outcome_count DESC;')

# Display the Results
landing_outcomes = pd.DataFrame(landing_outcomes)
landing_outcomes

,0,1
0,No attempt,10
1,Success (drone ship),6
2,Failure (drone ship),5
3,Success (ground pad),3
4,Controlled (ocean),3
5,Uncontrolled (ocean),2
6,Failure (parachute),2
7,Precluded (drone ship),1


### Reference Links

[Hands-on Lab : String Patterns, Sorting and Grouping](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org)  

[Hands-on Lab: Built-in functions](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org)

[Hands-on Lab : Sub-queries and Nested SELECT Statements](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org)

[Hands-on Tutorial: Accessing Databases with SQL magic](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb)

[Hands-on Lab: Analyzing a real World Data Set](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb)


This Jupyter Notebook was Completed By [Jonathan Scott](http://www.linkedin.com/in/jonathan-scott-140709317) as part of the IBM Data Science Certification.

## Author

Lakshmi Holla

## Other Contributors

Rav Ahuja

#### © IBM Corporation 2021. All rights reserved.